In [ ]:
#| default_exp utils.constraints

# Parameter constraints

> In this notebook I will try to implement weight constraint that keeps a specific set of paremeters positive.

The idea is basically traversing the weights `PyTree` and applying a function over the weights, which souldn't be specifically hard given we have `jax.tree_util.tree_map`. Then we can add that to a usual training loop to ensure that we can keep reaplying the same constraint during training to, for example, keep a specific set of weights above 0 at all times.

> Actually, we ended up using `flax.traverse_util.path_aware_map` because it keeps the names of the parent nodes so that we can check if we're inside a specific layer or not.

In [ ]:
import tensorflow as tf
tf.config.set_visible_devices([], device_type='GPU')

2023-05-06 16:42:08.061274: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-06 16:42:08.111834: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 16:42:10.456447: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
#| export
from jax import numpy as jnp
import flax
from flax.core import freeze

In [ ]:
from typing import Any, Callable, Sequence, Union
import numpy as np

import jax
from jax import lax, random, numpy as jnp
import flax
from flax.core import freeze, unfreeze, FrozenDict
from flax import linen as nn
from flax import struct
from flax.training import train_state

import optax

from clu import metrics
from ml_collections import ConfigDict

from einops import reduce

from iqadatasets.datasets import *
from fxlayers.layers import *

## Load the data

> We're going to employ `iqadatasets` to ease the loading of the data.

In [ ]:
dst_train = TID2008("/media/disk/databases/BBDD_video_image/Image_Quality/TID/TID2008/")
dst_val = TID2013("/media/disk/databases/BBDD_video_image/Image_Quality/TID/TID2013/")

In [ ]:
img, img_dist, mos = next(iter(dst_train.dataset))
img.shape, img_dist.shape, mos.shape

2023-05-06 16:42:16.513531: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


(TensorShape([384, 512, 3]), TensorShape([384, 512, 3]), TensorShape([]))

In [ ]:
img, img_dist, mos = next(iter(dst_val.dataset))
img.shape, img_dist.shape, mos.shape

2023-05-06 16:42:16.806541: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


(TensorShape([384, 512, 3]), TensorShape([384, 512, 3]), TensorShape([]))

In [ ]:
config = {
    "BATCH_SIZE": 32,
    "EPOCHS": 5,
    "LEARNING_RATE": 3e-4,
}
config = ConfigDict(config)
config

BATCH_SIZE: 32
EPOCHS: 5
LEARNING_RATE: 0.0003

In [ ]:
dst_train_rdy = dst_train.dataset.batch(config.BATCH_SIZE, drop_remainder=True)
dst_val_rdy = dst_val.dataset.batch(config.BATCH_SIZE, drop_remainder=True)

## Define the model we're going to use

> It's going to be a very simple model just for demonstration purposes.

In [ ]:
class GDN(nn.Module):
    """Generalized Divisive Normalization."""
    kernel_size: Union[int, Sequence[int]]
    strides: int = 1
    padding: str = "SAME"
    apply_independently: bool = False
    # feature_group_count: int = 1
    kernel_init: Callable = nn.initializers.lecun_normal()
    bias_init: Callable = nn.initializers.ones_init()
    alpha: float = 2.
    epsilon: float = 1/2 # Exponential of the denominator
    eps: float = 1e-6 # Numerical stability in the denominator

    @nn.compact
    def __call__(self,
                 inputs,
                 ):
        denom = nn.Conv(features=inputs.shape[-1], # Same output channels as input
                        kernel_size=self.kernel_size if isinstance(self.kernel_size, Sequence) else [self.kernel_size]*2, 
                        strides=self.strides, 
                        padding=self.padding,
                        # feature_group_count=self.feature_group_count,
                        feature_group_count=inputs.shape[-1] if self.apply_independently else 1,
                        kernel_init=self.kernel_init, 
                        bias_init=nn.initializers.constant(-1))(inputs**self.alpha)
        # denom = nn.relu(denom)
        return inputs / (denom**self.epsilon + self.eps)

In [ ]:
class PerceptNet(nn.Module):
    """IQA model inspired by the visual system."""

    @nn.compact
    def __call__(self,
                 inputs,
                 **kwargs,
                 ):
        outputs = nn.Conv(features=3, kernel_size=(5,5), strides=1, padding="SAME")(inputs)
        outputs = GDN(kernel_size=1, strides=1, padding="SAME", apply_independently=True)(outputs)
        outputs = nn.max_pool(outputs, window_shape=(2,2), strides=(2,2))
        outputs = nn.Conv(features=6, kernel_size=(5,5), strides=1, padding="SAME")(outputs)
        outputs = GDN(kernel_size=1, strides=1, padding="SAME", apply_independently=False)(outputs)
        outputs = nn.max_pool(outputs, window_shape=(2,2), strides=(2,2))
        # outputs = GaborLayer(features=128, kernel_size=21, strides=1, padding="SAME", fs=21)(outputs, **kwargs)
        outputs = nn.Conv(features=128, kernel_size=(5,5), strides=1, padding="SAME")(outputs)
        outputs = GDN(kernel_size=1, strides=1, padding="SAME", apply_independently=False)(outputs)
        return outputs

## Define the metrics with `clu`

In [ ]:
@struct.dataclass
class Metrics(metrics.Collection):
    """Collection of metrics to be tracked during training."""
    loss: metrics.Average.from_output("loss")

By default, `TrainState` doesn't include metrics, but it's very easy to subclass it so that it does:

In [ ]:
class TrainState(train_state.TrainState):
    metrics: Metrics
    state: FrozenDict

We'll define a function that initializes the `TrainState` from a module, a rng key and some optimizer:

In [ ]:
def create_train_state(module, key, tx, input_shape):
    """Creates the initial `TrainState`."""
    variables = module.init(key, jnp.ones(input_shape))
    state, params = variables.pop('params')
    return TrainState.create(
        apply_fn=module.apply,
        params=params,
        state=state,
        tx=tx,
        metrics=Metrics.empty()
    )

## Defining the training step

> We want to write a function that takes the `TrainState` and a batch of data can performs an optimization step.

In [ ]:
def pearson_correlation(vec1, vec2):
    vec1 = vec1.squeeze()
    vec2 = vec2.squeeze()
    vec1_mean = vec1.mean()
    vec2_mean = vec2.mean()
    num = vec1-vec1_mean
    num *= vec2-vec2_mean
    num = num.sum()
    denom = jnp.sqrt(jnp.sum((vec1-vec1_mean)**2))
    denom *= jnp.sqrt(jnp.sum((vec2-vec2_mean)**2))
    return num/denom

In [ ]:
@jax.jit
def train_step(state, batch):
    """Train for a single step."""
    img, img_dist, mos = batch
    def loss_fn(params):
        ## Forward pass through the model
        img_pred, updated_state = state.apply_fn({"params": params, **state.state}, img, mutable=list(state.state.keys()), train=True)
        img_dist_pred, updated_state = state.apply_fn({"params": params, **state.state}, img_dist, mutable=list(state.state.keys()), train=True)

        ## Calculate the distance
        dist = ((img_pred - img_dist_pred)**2).sum(axis=(1,2,3))**(1/2)
        
        ## Calculate pearson correlation
        return pearson_correlation(dist, mos), updated_state
    
    (loss, updated_state), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params)
    state = state.apply_gradients(grads=grads)
    metrics_updates = state.metrics.single_from_model_output(loss=loss)
    metrics = state.metrics.merge(metrics_updates)
    state = state.replace(metrics=metrics)
    state = state.replace(state=updated_state)
    return state

In their example, they don't calculate the metrics at the same time. I think it is kind of a waste because it means having to perform a new forward pass, but we'll follow as of now. Let's define a function to perform metric calculation:

In [ ]:
@jax.jit
def compute_metrics(*, state, batch):
    """Obtaining the metrics for a given batch."""
    img, img_dist, mos = batch
    def loss_fn(params):
        ## Forward pass through the model
        img_pred, updated_state = state.apply_fn({"params": params, **state.state}, img, mutable=list(state.state.keys()), train=True)
        img_dist_pred, updated_state = state.apply_fn({"params": params, **state.state}, img_dist, mutable=list(state.state.keys()), train=True)

        ## Calculate the distance
        dist = ((img_pred - img_dist_pred)**2).sum(axis=(1,2,3))**(1/2)
        
        ## Calculate pearson correlation
        return pearson_correlation(dist, mos)
    
    metrics_updates = state.metrics.single_from_model_output(loss=loss_fn(state.params))
    metrics = state.metrics.merge(metrics_updates)
    state = state.replace(metrics=metrics)
    return state

## Train the model!

First we'll instantiate the `TrainState` to get our parameters `PyTree`:

In [ ]:
state = create_train_state(PerceptNet(), random.PRNGKey(0), optax.adam(config.LEARNING_RATE), input_shape=(1,384,512,3))

And now we can apply `flax.traverse_util.path_aware_map()` together with `jnp.clip()` to ensure that there are no negative numbers in our GDN parameters:

In [ ]:
clipped_params = flax.traverse_util.path_aware_map(lambda path, v: jnp.clip(v, 0) if "GDN" in "_".join(path) else v, state.params)
clipped_params.keys()

dict_keys(['Conv_0', 'GDN_0', 'Conv_1', 'GDN_1', 'Conv_2', 'GDN_2'])

In [ ]:
state.params["GDN_1"]["Conv_0"], clipped_params["GDN_1"]["Conv_0"]

(FrozenDict({
     kernel: Array([[[[-0.20927475, -0.8931865 , -0.01957742,  0.39708266,
                0.57847786,  0.4533468 ],
              [ 0.06100697, -0.28133187,  0.50638074, -0.0699495 ,
               -0.6131419 , -0.25098443],
              [-0.38723496, -0.37814805,  0.07733525,  0.68892664,
               -0.2935091 ,  0.3425824 ],
              [-0.15717277, -0.65036553,  0.5565868 , -0.20142964,
               -0.10007945,  0.19361018],
              [-0.29459316, -0.65032095, -0.30642185, -0.52035373,
                0.00798638,  0.73876256],
              [-0.11728425, -0.7334936 , -0.70490634, -0.28179142,
               -0.2679925 ,  0.08794045]]]], dtype=float32),
     bias: Array([-1., -1., -1., -1., -1., -1.], dtype=float32),
 }),
 {'kernel': Array([[[[0.        , 0.        , 0.        , 0.39708266, 0.57847786,
            0.4533468 ],
           [0.06100697, 0.        , 0.50638074, 0.        , 0.        ,
            0.        ],
           [0.        , 0.     

Actually, our procedure will clip values both in `kernel` and in `bias`. If we want to restrict ourselves to only the kernel, we can do the following:

In [ ]:
clipped_params_kernel = flax.traverse_util.path_aware_map(lambda path, v: jnp.clip(v, 0) if ("GDN" in "_".join(path)) and ("kernel" in path) else v, state.params)
clipped_params_kernel.keys()

dict_keys(['Conv_0', 'GDN_0', 'Conv_1', 'GDN_1', 'Conv_2', 'GDN_2'])

In [ ]:
state.params["GDN_1"]["Conv_0"], clipped_params_kernel["GDN_1"]["Conv_0"]

(FrozenDict({
     kernel: Array([[[[-0.20927475, -0.8931865 , -0.01957742,  0.39708266,
                0.57847786,  0.4533468 ],
              [ 0.06100697, -0.28133187,  0.50638074, -0.0699495 ,
               -0.6131419 , -0.25098443],
              [-0.38723496, -0.37814805,  0.07733525,  0.68892664,
               -0.2935091 ,  0.3425824 ],
              [-0.15717277, -0.65036553,  0.5565868 , -0.20142964,
               -0.10007945,  0.19361018],
              [-0.29459316, -0.65032095, -0.30642185, -0.52035373,
                0.00798638,  0.73876256],
              [-0.11728425, -0.7334936 , -0.70490634, -0.28179142,
               -0.2679925 ,  0.08794045]]]], dtype=float32),
     bias: Array([-1., -1., -1., -1., -1., -1.], dtype=float32),
 }),
 {'kernel': Array([[[[0.        , 0.        , 0.        , 0.39708266, 0.57847786,
            0.4533468 ],
           [0.06100697, 0.        , 0.50638074, 0.        , 0.        ,
            0.        ],
           [0.        , 0.     

If we want to apply this at each step of the training process, we can wrap everything inside a function and add a "constraint step" into our training loop:

In [ ]:
#| exports
def clip_layer_kernel(params, # PyTree containing the parameters to clip.
                      layer_name: str, # String indicating the name of the layer. It can be a generic like "Conv" or specific like "Conv_0". 
                      a_min=None, # Min value to clip to.
                      a_max=None, # Max value to clip to.
                      ): # Same PyTree as `params` but with the corresponding values clipped.
    return freeze(flax.traverse_util.path_aware_map(lambda path, v: jnp.clip(v, a_min, a_max) if (layer_name in "_".join(path)) and ("kernel" in path) else v, params))

In [ ]:
#| exports
def clip_layer(params, # PyTree containing the parameters to clip.
               layer_name: str, # String indicating the name of the layer. It can be a generic like "Conv" or specific like "Conv_0". 
               a_min=None, # Min value to clip to.
               a_max=None, # Max value to clip to.
               ): # Same PyTree as `params` but with the corresponding values clipped.
    return freeze(flax.traverse_util.path_aware_map(lambda path, v: jnp.clip(v, a_min, a_max) if layer_name in "_".join(path) else v, params))

In [ ]:
@jax.jit
def train_step(state, batch):
    """Train for a single step."""
    img, img_dist, mos = batch
    def loss_fn(params):
        ## Forward pass through the model
        img_pred, updated_state = state.apply_fn({"params": params, **state.state}, img, mutable=list(state.state.keys()), train=True)
        img_dist_pred, updated_state = state.apply_fn({"params": params, **state.state}, img_dist, mutable=list(state.state.keys()), train=True)

        ## Calculate the distance
        dist = ((img_pred - img_dist_pred)**2).sum(axis=(1,2,3))**(1/2)
        
        ## Calculate pearson correlation
        return pearson_correlation(dist, mos), updated_state
    
    (loss, updated_state), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params)
    state = state.apply_gradients(grads=grads)
    metrics_updates = state.metrics.single_from_model_output(loss=loss)
    metrics = state.metrics.merge(metrics_updates)
    state = state.replace(metrics=metrics)
    state = state.replace(state=updated_state)
    return state

## Train the model!

In [ ]:
class GDN(nn.Module):
    """Generalized Divisive Normalization."""
    kernel_size: Union[int, Sequence[int]]
    strides: int = 1
    padding: str = "SAME"
    apply_independently: bool = False
    # feature_group_count: int = 1
    kernel_init: Callable = nn.initializers.lecun_normal()
    bias_init: Callable = nn.initializers.ones_init()
    alpha: float = 2.
    epsilon: float = 1/2 # Exponential of the denominator
    eps: float = 1e-6 # Numerical stability in the denominator

    @nn.compact
    def __call__(self,
                 inputs,
                 ):
        denom = nn.Conv(features=inputs.shape[-1], # Same output channels as input
                        kernel_size=self.kernel_size if isinstance(self.kernel_size, Sequence) else [self.kernel_size]*2, 
                        strides=self.strides, 
                        padding=self.padding,
                        # feature_group_count=self.feature_group_count,
                        feature_group_count=inputs.shape[-1] if self.apply_independently else 1,
                        kernel_init=self.kernel_init, 
                        bias_init=nn.initializers.ones)(inputs**self.alpha)
        # denom = nn.relu(denom)
        return inputs / (denom**self.epsilon + self.eps)

In [ ]:
state = create_train_state(PerceptNet(), random.PRNGKey(0), optax.adam(config.LEARNING_RATE), input_shape=(1,384,512,3))
state = state.replace(params=clip_layer(state.params, "GDN", a_min=0))

In [ ]:
metrics_history = {
    "train_loss": [],
    "val_loss": [],
}

In [ ]:
%%time
for epoch in range(config.EPOCHS):
    ## Training
    for batch in dst_train_rdy.as_numpy_iterator():
        state = train_step(state, batch)
        state = state.replace(params=clip_layer(state.params, "GDN", a_min=0))
        # state = compute_metrics(state=state, batch=batch)
        break

    ## Log the metrics
    for name, value in state.metrics.compute().items():
        metrics_history[f"train_{name}"].append(value)
    
    ## Empty the metrics
    state = state.replace(metrics=state.metrics.empty())

    ## Evaluation
    for batch in dst_val_rdy.as_numpy_iterator():
        state = compute_metrics(state=state, batch=batch)
        break
    for name, value in state.metrics.compute().items():
        metrics_history[f"val_{name}"].append(value)
    state = state.replace(metrics=state.metrics.empty())
    
    print(f'Epoch {epoch} -> [Train] Loss: {metrics_history["train_loss"][-1]} [Val] Loss: {metrics_history["val_loss"][-1]}')
    break

Epoch 0 -> [Train] Loss: -0.6268874406814575 [Val] Loss: -0.6873180866241455
CPU times: user 14.6 s, sys: 6.73 s, total: 21.3 s
Wall time: 3.82 s
